In [9]:
# Dependencies
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import requests
import json
import alpha_vantage
from config import apiKey
from alpha_vantage.timeseries import TimeSeries

def one_year_data(stock_ticker): 
    yesterday = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
    past_year = datetime.strftime(datetime.now() - timedelta(366), '%Y-%m-%d')
    

    url = "https://www.alphavantage.co/query?"
    query_url = f"{url}function=TIME_SERIES_DAILY&symbol={stock_ticker}&outputsize=full&apikey={apiKey}" 

    response = requests.get(query_url).json() 

    df = pd.DataFrame(response["Time Series (Daily)"]) 

    df_transposed = df.T # or df1.transpose()

    df_filtered = df_transposed[yesterday : past_year] 

    df_filtered.rename(columns={'4. close':'close', '3. low':'low', '2. high':'high', '5. volume':'volume'}, inplace=True)
    
    df_filtered.drop('1. open', axis=1, inplace=True)
    
    df_filtered = df_filtered[["close", "low", "high", "volume"]]
    
    
    return df_filtered  

one_year_data("AAPL")    

/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/addisonpietroburgo/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,close,low,high,volume
2019-03-22,191.0500,190.7800,197.6900,42269598
2019-03-21,195.0900,189.8100,196.3300,51034237
2019-03-20,188.1600,184.7300,189.4900,31035231
2019-03-19,186.5300,185.9200,188.9900,31646369
2019-03-18,188.0200,185.7900,188.3900,26219832
2019-03-15,186.1200,183.7400,187.3300,39042912
2019-03-14,183.7300,182.5600,184.1000,23579508
2019-03-13,181.7100,180.9200,183.3000,31032524
2019-03-12,180.9100,179.3700,182.6700,32467584
2019-03-11,178.9000,175.3500,179.1200,32011034
